### Setup

In [29]:
# gen imports
import pandas as pd, numpy as np, os, pyabc
os.chdir("C:/Users/jstelman/Git/chelsvig_urban_pesticides/src/")   # for small laptop
# os.chdir("C:/Git/chelsvig_urban_pesticides/src/")   # For Desktop
from path_names import *
from prpy_bookkeeping import *
nsims = 5
Ite, i = 1, 1

### Defining a model

(From "quickstart" example from pyabc docs)
To do model selection, we first need some models. A model, in the simplest case,
is just a callable which takes a single `dict` as input and returns a single `dict` as output. The keys of the input dictionary are the parameters of the model, the output
keys denote the summary statistics.


In [2]:
"""
Function to take in parameters (in dictionary form) and output summary statistics
 Input: 
  SWMM parameters:
    (see explanations here: <https://docs.google.com/document/d/1pVnjSP3cCKBCWUV3Brns8hJpkgHLekXns_9WG-qla7E/>)
    NImperv
    NPerv
    SImperv
    SPerv
    PctZero
    MaxRate
    MinRate
    Decay
    DryTime
    Por
    WP
    FC
    Ksat
    Roughness
    Kdecay
    BCoeff2
    WCoeff2
  
  VVWM parameters:
    (see explanations here: <https://docs.google.com/document/d/1JM8guprnugzJDvCaBwD-HCpgR5NvTGnCe3qjFXhMQHU/>)
    kd - Sorption coefficient (mL/g) -
    aer_aq - Water column degradation half-life (days) -
    aer_aq_temp - Reference temp. For water column degradation (C) -
    anae_aq - Benthic degradation half-life (days) -
    anae_aq_temp - Reference temp. For benthic degradation (C) -
    photo - Photolysis half-life (days) -
    rflat - Reference latitude for photolysis -
    hydro - Hydrolysis half-life (days) -
    mwt - Molecular weight  -
    vapor - Vapor pressure (torr) -
    sol - Solubility (mg/L) -
    henry - Henry’s law constant -
    benthic_depth - Depth of benthic region (m) -
    porosity - Porosity of benthic region (--) -
    bulk_density - Bulk density of benthic region -
    froc2 - Fraction of organic carbon on sediment in benthic region -
    doc2 - Concentration of dissolved organic carbon in benthic region (mg/L) -
    bnmas - Areal concentration of biosolids in benthic region (g/m2) -
    sused - Suspended solids concentration in water column (mg/L) -
    chl - Chlorophyll concentration in water column (mg/L) -
    froc1 - Fraction of organic carbon on suspended sediment in water column -
    doc1 - Concentration of dissolved organic carbon in water column (mg/L) -
    area - Area of water body (m2) -

 Output:
  The values of 106 date- and site- specific bifenthrin concentrations (keys are date/site code indicators)
  Why? Because that's the observation data we have access to
"""
def model(parameters: dict) -> dict:
    """
    turn swmm parameters into one csv file. 
    turn vvwm parameters into another csv file.
    if any of the the parameters are not okay:
        (that means max and min and FC and WP relationships follow rules and that everything else is in the bounds we need)
        return a dictionary full of nans and don't do the next part.
    else:
        run python scripts 03-06 and 08-09 (i/Ite = c)
        run some derivation of postprocess notebook to take 106 cells of that file made by 09 and turn it into a dictionary
    return (the dictionary)
    """
    # turn swmm parameters into one csv file

### File 02 would be here, but actually, it will take place in the prior segment

In [91]:
"""
Simulate dict of LHS parameters from the first row of the dfs of swmm parameters and of vvwm parameters
"""
lhs_design = pd.concat([pd.read_csv(os.path.join(dir_path, "io", "lhs_sampled_params.csv"), index_col=0),
                        pd.read_csv(os.path.join(dir_path, "io", "lhs_sampled_params_vvwm.csv"), index_col=0)],
                      axis = 1)
# round lhs decimals
lhs_design = lhs_design.round(
    {   # swmm
        "NImperv": 4, "NPerv": 4, "SImperv":3, "SPerv": 3, "PctZero": 3, "MaxRate": 3, "MinRate": 3, "Decay": 2, 
        "DryTime": 0, "Por": 3, "WP": 3, "FC": 3, "Ksat": 3, "Rough": 4, "Kdecay": 3, "BCoeff2": 3, "WCoeff2": 3,
        # vvwm
        "kd": 2, "aer_aq": 0, "aer_aq_temp": 2, "anae_aq": 0, "anae_aq_temp": 2, "photo": 2, "rflat": 0, "hydro": 2, 
        "sol": 4, "benthic_depth": 3, "porosity": 2, "bulk_density": 2, "froc2": 3, "doc2": 2, "bnmas": 3, "sused": 3, 
        "chl": 3, "froc1": 3, "doc1": 2
    })
# SWMM lhs parameter (1 row) as dictionary
params1 = lhs_design.to_dict(orient = 'records')[0]   

In [88]:
pd.concat([pd.read_csv(os.path.join(dir_path, "io", "lhs_sampled_params.csv"), index_col=0),
                        pd.read_csv(os.path.join(dir_path, "io", "lhs_sampled_params_vvwm.csv"), index_col=0)],
                      axis = 1)

,NImperv,NPerv,SImperv,SPerv,PctZero,MaxRate,MinRate,Decay,DryTime,Por,...,benthic_depth,porosity,bulk_density,froc2,doc2,bnmas,sused,chl,froc1,doc1
0,0.017880,0.395095,1.362840,4.441707,15.199846,81.273279,7.266082,6.264549,8.185651,0.465198,...,0.883951,0.705439,1.608749,0.020257,27.615951,1.093691,51.268912,0.983770,0.113057,0.638653
1,0.023986,0.480700,2.435573,3.761971,95.290006,66.208890,0.463204,4.041612,12.057423,0.429331,...,0.484670,0.230943,1.546444,0.019234,26.459817,0.343051,16.552345,0.131727,0.074840,0.338567
2,0.019504,0.422362,2.413312,4.307667,33.529851,45.622883,2.983237,5.499093,10.307100,0.439336,...,0.173770,0.299145,1.096895,0.026357,52.470446,2.732693,8.728867,1.094471,0.082283,0.042302
3,0.010947,0.418649,1.762243,2.547608,55.905716,22.502936,2.632963,5.095740,2.179982,0.423847,...,0.693997,0.620549,1.607612,0.008449,4.646844,1.298799,13.139179,0.450476,0.079333,0.832264
4,0.020289,0.172024,2.156471,4.337046,7.964026,37.181238,7.233648,3.304149,11.580703,0.406709,...,0.786336,0.414257,1.462563,0.024717,56.523490,4.973299,3.866159,1.322423,0.075184,0.445047


In [83]:
"""
Simulate a row of df LHS parameters 
"""
#### from 08 {
lhs_design = pd.read_csv(os.path.join(dir_path, "io", "lhs_sampled_params_vvwm.csv"), index_col=0)
# round lhs decimals
lhs_design = lhs_design.round(
    {"kd": 2, "aer_aq": 0, "aer_aq_temp": 2, "anae_aq": 0, "anae_aq_temp": 2, "photo": 2, "rflat": 0, "hydro": 2, "sol": 4,
     "benthic_depth": 3, "porosity": 2, "bulk_density": 2, "froc2": 3, "doc2": 2, "bnmas": 3, "sused": 3, "chl": 3,
     "froc1": 3, "doc1": 2})#### }
# SWMM lhs parameter (1 row) as dictionary
vvwm_params = lhs_design.to_dict(orient = 'records')[0]   

In [92]:
params1

{'NImperv': 0.0179,
 'NPerv': 0.3951,
 'SImperv': 1.363,
 'SPerv': 4.442,
 'PctZero': 15.2,
 'MaxRate': 81.273,
 'MinRate': 7.266,
 'Decay': 6.26,
 'DryTime': 8.0,
 'Por': 0.465,
 'FC': 0.361,
 'WP': 0.119,
 'Ksat': 1.719,
 'Rough': 0.0194,
 'Kdecay': 0.045,
 'BCoeff2': 1.584,
 'WCoeff2': 0.203,
 'kd': 5333.21,
 'aer_aq': 272.0,
 'aer_aq_temp': 22.96,
 'anae_aq': 255.0,
 'anae_aq_temp': 23.76,
 'photo': 404.98,
 'rflat': 40.0,
 'hydro': 106.3,
 'sol': 0.48,
 'benthic_depth': 0.884,
 'porosity': 0.71,
 'bulk_density': 1.61,
 'froc2': 0.02,
 'doc2': 27.62,
 'bnmas': 1.094,
 'sused': 51.269,
 'chl': 0.984,
 'froc1': 0.113,
 'doc1': 0.64}

In [103]:
'''
Then we would have to split them into two after they have been passed into the function
'''
swmm_keys = list(params1.keys())[:17]
vvwm_keys = list(params1.keys())[17:]
swmm_params = {key: params1[key] for key in swmm_keys}
vvwm_params = {key: params1[key] for key in vvwm_keys}

In [106]:
vvwm_params

{'kd': 5333.21,
 'aer_aq': 272.0,
 'aer_aq_temp': 22.96,
 'anae_aq': 255.0,
 'anae_aq_temp': 23.76,
 'photo': 404.98,
 'rflat': 40.0,
 'hydro': 106.3,
 'sol': 0.48,
 'benthic_depth': 0.884,
 'porosity': 0.71,
 'bulk_density': 1.61,
 'froc2': 0.02,
 'doc2': 27.62,
 'bnmas': 1.094,
 'sused': 51.269,
 'chl': 0.984,
 'froc1': 0.113,
 'doc1': 0.64}

### From 03

In [ ]:
import pytest_shutil, shutil, regex as re, dask
loginfo = log_prefixer("03")

In [107]:
# Helper Functions

'''
Helper function of edit_file:
Create new line of .inp file with lhs simulated versions of values and text in old .inp file
 Inputs: fileline <str> -line from original .inp file-
   Col <int> -column in the .inp file that contains the info you want to preserve and clean- 
   sim <float> -the simulated value to repace the old (observed) one with-
 Output: newline <str> -custom version of line for new file-
'''
def edit1line(fileline, Col, sim):
    listline = fileline.split()
    listline[Col] = str(sim)
    newline = ' '.join([str(item) for item in listline]) + "\n"
    return(newline)

# From 03 with edits

'''
Edit the new file to be cleaner
 Inputs: Ite <int> -index of current simulation-
   Num <int> -Number of rows to clean-
   row_0 <int> -Number of rows to skip-
   parameter <str> -name of lhs design parameter (will become name of column in the .csv  file)-
   Col <int> -column in the .inp file that contains the info you want to preserve and clean- 
   flines <list of str> -lines of the file to clean up-
 Output: cleaned up lines of file given
'''
def editted_lines(swmm_dict, Num, row_0, parameter, Col, flines):
    # value of "parameter" in current lhs simulation
    # sim = lhs_design.loc[Ite - 1, parameter]   # updating
    sim = swmm_dict[parameter]
    print(sim)
    return([edit1line(flines[row_0 + i], Col, sim) for i in range(Num)])


In [ ]:
'''
Step 1:
Gernerate random id (see "uuid.uuid4().hex[0:8]     #generate random jobID" in Jeff's code on bookmarks bar)
Make input and output file names incorporate this id (so things don't get lost or overwritten thanks to parallelism. Yay)
Also make DLL copy incorporate this id 

Step 2:
Run the thing!
Run swmm toolbox on it
Then delete that massive output file and the dll.
Honestly, maybe even the input file. 

Step 3:
Carry on with the rest of the munging til the VVWM step. 

Step 4:
Do something similar to step 1 for VVWM
'''

In [19]:
"""
Simulate the first step of running the model for one iteration
"""
# From 03 - with a few edits

# make the folder, we can probably skip this,
# we should probably look into the tempfile package
new_dir = os.path.join(swmm_path, "input_" + str(Ite))

if not os.path.exists(new_dir):
    os.mkdir(new_dir)
    print("Folder ", Ite, " created", "\n")
else:
    print("Folder ", Ite, "already exists")

# copy base file into new file location
old_path = os.path.join(swmm_path, "NPlesantCreek.inp")
# RIGHT HERE!!!!!!!! temp it instead.... but what to do with the output?
new_path = os.path.join(new_dir, "NPlesantCreek.inp")
# loginfo("Copying base swmm input file <" + old_path + "> into <" + new_dir + ">.")
shutil.copyfile(old_path, new_path)

# start reading the new file
# loginfo("Opening file <" + new_path + "> to read.")
new_file = open(new_path, "r")
filelines = new_file.readlines()
# loginfo("Closing file <" + new_path + "> after reading lines into list.")
new_file.close()

# edit the new file

# -----------------------------
# first we need to correct some absolute paths, because they are currently only set to work on the author's computer
filelines = replace_infile_abspaths(filelines = filelines)

# ---------------------------
# 113 = number of subcatchments

# parameter = NImperv
filelines[172:(172 + 113)] = editted_lines(swmm_dict = swmm_params, Num = 113, row_0 = 172, parameter = "NImperv", Col = 1, flines = filelines)

# parameter = NPerv
filelines[172:(172 + 113)] = editted_lines(swmm_dict = swmm_params, Num = 113, row_0 = 172, parameter = "NPerv", Col = 2, flines = filelines)

# parameter = SImperv
filelines[172:(172 + 113)] = editted_lines(swmm_dict = swmm_params, Num = 113, row_0 = 172, parameter = "SImperv", Col = 3, flines = filelines)

# parameter = SPerv
filelines[172:(172 + 113)] = editted_lines(swmm_dict = swmm_params, Num = 113, row_0 = 172, parameter = "SPerv", Col = 4, flines = filelines)

# parameter = PctZero
filelines[172:(172 + 113)] = editted_lines(swmm_dict = swmm_params, Num = 113, row_0 = 172, parameter = "PctZero", Col = 5, flines = filelines)

# parameter = MaxRate
filelines[289:(289 + 113)] = editted_lines(swmm_dict = swmm_params, Num = 113, row_0 = 289, parameter = "MaxRate", Col = 1, flines = filelines)

# parameter = MinRate
filelines[289:(289 + 113)] = editted_lines(swmm_dict = swmm_params, Num = 113, row_0 = 289, parameter = "MinRate", Col = 2, flines = filelines)

# parameter = Decay
filelines[289:(289 + 113)] = editted_lines(swmm_dict = swmm_params, Num = 113, row_0 = 289, parameter = "Decay", Col = 3, flines = filelines)

# parameter = DryTime
filelines[289:(289 + 113)] = editted_lines(swmm_dict = swmm_params, Num = 113, row_0 = 289, parameter = "DryTime", Col = 4, flines = filelines)
# ---------------------------

# ---------------------------
# 1 = number of aquifers

# parameter = Por
filelines[406:(406 + 1)] = editted_lines(swmm_dict = swmm_params, Num = 1, row_0 = 406, parameter = "Por", Col = 1, flines = filelines)

# parameter = WP
filelines[406:(406 + 1)] = editted_lines(swmm_dict = swmm_params, Num = 1, row_0 = 406, parameter = "WP", Col = 2, flines = filelines)

# parameter = FC
filelines[406:(406 + 1)] = editted_lines(swmm_dict = swmm_params, Num = 1, row_0 = 406, parameter = "FC", Col = 3, flines = filelines)

# parameter = Ksat
filelines[406:(406 + 1)] = editted_lines(swmm_dict = swmm_params, Num = 1, row_0 = 406, parameter = "Ksat", Col = 4, flines = filelines)
# ---------------------------

# ---------------------------
# 195 = number of conduits

# # parameter = Rough
# filelines[734:(734 + 195)] = editted_lines(swmm_dict = swmm_params, Num = 195, row_0 = 734, parameter = "Rough", Col = 4, flines = filelines)
# ---------------------------

# ---------------------------
# 1 = number of pollutants

# parameter = Kdecay
filelines[1125:(1125 + 1)] = editted_lines(swmm_dict = swmm_params, Num = 1, row_0 = 1125, parameter = "Kdecay", Col = 5, flines = filelines)

# parameter = BCoeff2
filelines[1371:(1371 + 1)] = editted_lines(swmm_dict = swmm_params, Num = 1, row_0 = 1371, parameter = "BCoeff2", Col = 4, flines = filelines)

# parameter = WCoeff2
filelines[1377:(1377 + 1)] = editted_lines(swmm_dict = swmm_params, Num = 1, row_0 = 1377, parameter = "WCoeff2", Col = 4, flines = filelines)
# ---------------------------   

# copy, write out file
loginfo("Opening file <" + new_path + "> to overwrite with edited content.")
new_file = open(new_path, "w")
new_file.writelines(filelines)
# loginfo("Closing file <" + new_path + ">.")
new_file.close()


### From 04

In [ ]:
from pyswmm import Simulation
import swmmtoolbox.swmmtoolbox as swmmtoolbox
import time
from pyswmm.lib import DLL_SELECTION
loginfo = log_prefixer("04")
inp_dir_prefix = os.path.join(dir_path, "input", "swmm", "input_")

In [ ]:
# Make DLL folder

# save the path for the original dynamic link library
dll_path = DLL_SELECTION()
# save its base name (just the name of the file)
dll_bn = os.path.basename(dll_path)
# save the path to a new folder where copies of dll will be stored upon creation
dll_dir = os.path.join(dir_path, "input", "swmm", "dll")
# make sure that folder exists. If it doesn't, create it.
if not os.path.exists(dll_dir):
    loginfo("Creating directory <" + dll_dir + ">.")
    print("Creating <dll/> directory.")
    os.mkdir(dll_dir)

In [ ]:
# log something generic
loginfo("Simmulation " + str(i) + " of " + str(nsims))

# create path to the 1-time-use dll file we are going to create
dll_i = dll_bn[:dll_bn.rindex(".")] + '-' + str(i) + dll_bn[dll_bn.rindex("."):]
lib_path = os.path.join(dll_dir, dll_i)
# create 1-time-use dll file copy
shutil.copyfile(dll_path, lib_path)

# specify the directory with the file pyswmm needs by attaching the folder id to the rest of the folder's absolute path
sim_dir = inp_dir_prefix + str(i)
# specify the actual file pyswmm needs
sim_path = os.path.join(sim_dir, r'NPlesantCreek.inp')
#print("Simulation input file found:", sim_path)

# specify the file that pyswmm will (over)write with output after running the probabilistic simulation
sim_bin_path = os.path.join(sim_dir, "NPlesantCreek.out")
# delete pre-existing .out, if present, in order to run swmm agreeably
if os.path.exists(sim_bin_path):
    loginfo("Deleting current copy of <" + sim_bin_path + "> so new copy can be created.")
    #print("Deleting current copy of <NPlesantCreek.out> so new copy can be created.")
    os.remove(sim_bin_path)

# stagger starting times 1 sec apart
time.sleep(i)

# load the model {no interaction, write (binary) results to <JS_NPlesantCreek.out>, use the specified dll}
sim = Simulation(inputfile=sim_path, reportfile=None, outputfile=sim_bin_path, swmm_lib_path=lib_path)
# simulate the loaded model
loginfo("Executing SWMM simmulation with no interaction. Input from <" + sim_path + ">. Will store output in <" + sim_bin_path + ">.")
# sim.execute()
with sim as s:
    for step in s:
        pass

# extract swmm outputs with swmmtoolbox and delete expensive binary files
lab1 = 'subcatchment,,Runoff_rate'
lab2 = 'subcatchment,,Bifenthrin'
runf_stack = swmmtoolbox.extract(sim_bin_path, lab1)
bif_stack = swmmtoolbox.extract(sim_bin_path, lab2)

loginfo("Deleting <" + sim_bin_path + "> to free up memory.")
os.remove(sim_bin_path)
loginfo("Deleting <" + dll_i + "> to free up memory.")
os.system("rm " + lib_path)
print("Deleted <input_" + str(i) + "/JS_NPlesantCreek.out> and <" + dll_i + "> to free up memory.")    

# compute and export daily averages to csv files and finish
runf_davg = runf_stack.resample('D').mean()
bif_davg = bif_stack.resample('D').mean()
# print(save_and_finish(runf_davg, os.path.join(sim_dir, "swmm_output_davg_runf.csv")))
# print(save_and_finish(bif_davg, os.path.join(sim_dir, "swmm_output_davg_bif.csv")))
# msg1 and msg2 text will be the same, but we must do both to save both csvs


### Now we need priors 

Not sure how to do that because there seems to be a much stricter method for prior declaration. 
Maybe I can do this first step outside
then enter it as a a normal distribution with a sigma of 0 for all the parameters that are special

In [79]:
"""
Priors. Get the values from that csv. Just for SWMM at first.
"""
swmm_ranges = pd.read_csv(os.path.join(dir_path, "input", "lhs", "lhs_param_ranges.csv"), index_col=0,
                           usecols = ["Parameter","Min", "Range"])

In [80]:
'''
Link up with the vvwm priors and make one big list with 36 params
'''
vvwm_ranges = pd.read_csv(os.path.join(dir_path, "input", "lhs", "lhs_param_ranges_vvwm.csv"), index_col=0,
                           usecols = ["Parameter","Min", "Range"])

param_ranges = pd.concat([swmm_ranges, vvwm_ranges], axis = 0)

priors = param_ranges.to_dict("index")

# borrowed from Jeff: <https://github.com/JeffreyMinucci/bee_neonic_abc/blob/master/pyabc_run.ipynb>
prior = pyabc.Distribution(**{key: pyabc.RV("uniform", loc = v['Min'], scale = v['Range'])
                        for key, v in priors.items()})
prior


<Distribution 'BCoeff2', 'Decay', 'DryTime', 'FC', 'Kdecay', 'Ksat', 'MaxRate', 'MinRate', 'NImperv', 'NPerv', 'PctZero', 'Por', 'Rough', 'SImperv', 'SPerv', 'WCoeff2', 'WP', 'aer_aq', 'aer_aq_temp', 'anae_aq', 'anae_aq_temp', 'benthic_depth', 'bnmas', 'bulk_density', 'chl', 'doc1', 'doc2', 'froc1', 'froc2', 'hydro', 'kd', 'photo', 'porosity', 'rflat', 'sol', 'sused'>

In [77]:
param_ranges

,Min,Range
Parameter,,
NImperv,0.010000,0.015000
NPerv,0.050000,0.450000
SImperv,1.270000,1.270000
SPerv,2.540000,2.540000
PctZero,0.010000,99.990000
MaxRate,8.460000,118.540000
MinRate,0.254000,10.666000
Decay,2.000000,5.000000
DryTime,2.000000,12.000000


### So then I could do something like this

In [ ]:
abc = ABCSMC(model, prior, distance = hydroeval.NSE)

### Then I need to set the observed data 

I will have to make that csv into a dictionary like how Jeff did it. 
This should be the easiest part
I've basically already done some of it in the post-processing notebook.